In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd
import p

In [2]:
pd.options.plotting.backend = "plotly"

## Connection to the Google Sheet

In [ ]:
# Read and Load Credentials
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/gabrielreynoso/Documents/GoogleCredentials/gabo_credentials.json')
gc = gspread.authorize(credentials)

## Connection to the DB

In [6]:
# DB Information
db_credentials = open('/Users/gabrielreynoso/Documents/Queries/db_klarprod_connection.txt', 'r')
# Read file
postgres_str = db_credentials.read()
# Close file
db_credentials.close()
cnx = create_engine(postgres_str)

## Read queries function

In [49]:
def read_queries_from_file(filename) -> list:
    """
    Open and read the file as a single buffer and returns a list of Dataframes with all the queries
    :param filename: Name of the file with the queries
    :return: List with Dataframes of executed queries
    """
    queries = {}
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    # All SQL commands (split on ';')
    queries_aux = sqlFile.split(';')
    num = len(queries_aux)//2
    for idx in range(num):
        name = queries_aux[idx].split("--")[1]
        queries[name] = queries_aux[idx+1]
    return queries

In [50]:
queries = read_queries_from_file('MBR_queries.sql')

{'Funnel View': "\nwith started_signup as (\n\twith started_signups as (\n\t select date_trunc('month', started_web_signup_mx) as mes,\n\t count(user_id) as started_signup\n\t from klar.cck_funnel cf\n\t where started_web_signup_mx notnull\n\t and started_web_signup_mx > '2021-04-01'\n\t group by mes\n\t order by 1 asc\n),\n\tnot_completed_signup as (\n\t     select date_trunc('month', signup_created_timestamp) as day_notC,\n\t           count(email) as not_c_signup\n\t     from klar.signup_not_completed\n\t     where signup_created_timestamp notnull\n\t     group by day_notC\n)\nselect day_notC as fecha,\n           not_c_signup + started_signup as signups_started\n    from not_completed_signup\n             left join started_signups on not_completed_signup.day_notC = started_signups.mes\n    order by fecha asc\n),\n\tcompleted_signup as (\n\tselect date_trunc('month', ended_web_signup_mx) as fecha,\n\tcount(user_id) as completed_signup\n\tfrom klar.cck_funnel cf2\n\twhere ended_web_s